En este cuaderno, sobre la base de datos de Habitaclia obtenida en 2024, obtendremos dos nuevas bases de datos para entrenar un modelo que nos acerque a medir la calidad a través del texto. 

1. Sólo texto - calidad del 1 al 10. 
2. Texto + enlace a fotos - calidad del 1 al 10.

In [1]:
import pandas as pd
import numpy as np

# Lista de participantes

In [2]:
nombres = [
    "Alsharif, Dareen",
    "Izumi Noda, Henry Daisuke",
    "Lopez Lopez, Víctor Stheffano",
    "Peralta Carvallo, Loreto Beatriz",
    "Román Mantilla, María Alejandra",
    "Ruiz Amezquita, Tatiana",
    "Ruiz Acha, Kathia",
    "Tian, Shiduo (Leo)",
    "Vásquez Umaña, José Daniel"
]

# Texto

In [3]:
df1=pd.read_excel('G:/Mi unidad/Documentos personales/9-Documentos Doctorado/Doctorado 2023/11-Web scrapping/data/a-rmb-marzo24.xlsx')
len(df1)

2899

In [4]:
df1.head(2)

,ID,Title,Link,precio_euros,codigo_inmueble,Description,last_mod,dummy_indice,indice_alquiler,area,...,parking,pool,lift,furniture,public_transp,barrio,codigo_inmueble_numero,Lat_Y,Lon_X,multifam
0,1,Alquiler Estudio en Collblanc,https://www.habitaclia.com/alquiler-estudio-co...,770,Anuncio: 48377000000745,RESERVAS E INQUIETUDESPara obtener información...,20/03/2024,0,0.0,25,...,0,0,0,0,0,Collblanc,48377000000745,41.372658,2.116238,1
1,2,Alquiler Piso Carrer del montseny,https://www.habitaclia.com/alquiler-piso-la_to...,1180,Anuncio: 29091000000788,DISPONIBLE DESDE EL 15 DE MAYO DE 2024========...,15/03/2024,1,25.0,49,...,0,0,0,0,0,La Torrassa,29091000000788,41.369947,2.119384,1


In [5]:
df1['Description'].isna().sum()

0

In [6]:
df1['Description'].duplicated().sum()

140

In [7]:
df1['cal_texto'] = None

Para no sesgar la calificación, vamos a darles diferentes muestras entre la evaluación del texto y de las imágenes. Para ello creamos dos IDs: `IDtext` y `IDimg`. Aquí creo el primero, que es exactamente igual al ID original del dataframe.

In [8]:
df1['IDtext']=df1['ID']

Y ahora asigno responsables.

In [9]:
# Ordenar la lista de nombres según la columna 'IDtext'
nombres_ordenados = sorted(nombres)

# Calcular el tamaño de cada parte
part_size = len(df1) // len(nombres_ordenados)
remainder = len(df1) % len(nombres_ordenados)

# Dividir df1 en partes
partes_df = np.split(df1.sample(frac=1).sort_values(by='IDtext'), np.cumsum([part_size] * (len(nombres_ordenados) - 1) + [part_size + remainder]))

# Asignar los nombres a cada parte
for i, parte in enumerate(partes_df):
    if i < len(nombres_ordenados):
        parte['Responsable texto'] = nombres_ordenados[i]
    else:
        parte['Responsable texto'] = nombres_ordenados[-1]  # Asignar el último nombre en caso de que haya más partes que nombres

# Concatenar las partes nuevamente en un solo DataFrame
df1 = pd.concat(partes_df)

La tabla se vería así.

In [10]:
df1[['IDtext','Description','cal_texto','Responsable texto']].head(3)

,IDtext,Description,cal_texto,Responsable texto
0,1,RESERVAS E INQUIETUDESPara obtener información...,None,"Alsharif, Dareen"
1,2,DISPONIBLE DESDE EL 15 DE MAYO DE 2024========...,None,"Alsharif, Dareen"
2,3,*El reportaje fotográfico es una combinación d...,None,"Alsharif, Dareen"


Por defecto ya está ordenado de menor a mayor según el `IDtext`.

In [12]:
df1[['IDtext','Description','cal_texto','Responsable texto']].to_excel('eva_tex-rmb-marzo24.xlsx', index=False)

Tras la revisión de la columna `Description` deberán indicar un valor del 1 al 10 en la columna `cal_texto` sobre la siguiente pregunta: *¿De qué tan buena calidad arquitectónica crees que es el piso en oferta?*. 

Si consideran que la descripción **no** ofrece suficiente información para dar un veredicto sobre la calidad del inmueble, indicar el valor 0.

# Imagen

In [13]:
df1['dum_ext']=None
df1['dum_render']=None
df1['dum_no_varios']=None
df1['dum_no_link']=None
df1['cal_img']=None

La columna `IDimg`, será el resultado de multiplicar por 2 los valores pares de `IDtext`, y por 3 los valores impares.

In [14]:
df1['IDimg'] = df1['IDtext'].apply(lambda x: x * 2 if x % 2 == 0 else x * 3)

Ordeno de menor a mayor con base en esa nueva columna para exportar.

In [15]:
df1 = df1.sort_values(by='IDimg')

Y ahora asigno responsables.

In [16]:
# Ordenar la lista de nombres según la columna 'IDtext'
nombres_ordenados = sorted(nombres)

# Calcular el tamaño de cada parte
part_size = len(df1) // len(nombres_ordenados)
remainder = len(df1) % len(nombres_ordenados)

# Dividir df1 en partes
partes_df = np.split(df1.sample(frac=1).sort_values(by='IDimg'), np.cumsum([part_size] * (len(nombres_ordenados) - 1) + [part_size + remainder]))

# Asignar los nombres a cada parte
for i, parte in enumerate(partes_df):
    if i < len(nombres_ordenados):
        parte['Responsable img'] = nombres_ordenados[i]
    else:
        parte['Responsable img'] = nombres_ordenados[-1]  # Asignar el último nombre en caso de que haya más partes que nombres

# Concatenar las partes nuevamente en un solo DataFrame
df1 = pd.concat(partes_df)

Para esta etapa deberán revisar las fotografías del interior del inmueble para completar la columna `cal_img` en  una escala del 1 al 10 con base en la siguiente pregunta: *¿De qué tan buena calidad arquitectónica crees que es el piso en oferta?*

Únicamente **los casos que no cumplan con tener al menos una fotografía del interior** deberán considerar las siguientes situaciones:
1. Si sólo hay fotos del exterior del inmueble, indicar 1 en la columna `dum_ext`. Si no es el caso, dejarla vacía.
2. Si sólo hay renders, indicar 1 en la columna `dum_render`. Si no es el caso, dejarla vacía.
3. Si hay una combinación de las dos situaciones anteriores, indicar 1 en la columna `dum_no_varios`. Si no es el caso, dejarla vacía.
4. Si el enlace está obsoleto indicar 1 en la columna `dum_no_link`. Si no es el caso, dejarla vacía.

Reviso la cabecera.

In [17]:
df1[['IDimg','cal_img','dum_ext','dum_render','dum_no_varios','dum_no_link','Responsable img']].head(3)

,IDimg,cal_img,dum_ext,dum_render,dum_no_varios,dum_no_link,Responsable img
0,3,None,None,None,None,None,"Alsharif, Dareen"
1,4,None,None,None,None,None,"Alsharif, Dareen"
3,8,None,None,None,None,None,"Alsharif, Dareen"


In [18]:
df1[['IDimg','Link','cal_img','dum_ext','dum_render',
     'dum_no_varios','dum_no_link','Responsable img']].to_excel(
    'eva_img-rmb-marzo24.xlsx',
    index=False)